In [1]:
import re
import os
import pandas as pd

def readfile(path):
    dirs = os.listdir(path)
    column_names = ['label', 'text']
    data_ = []
    data = []
    for d in dirs:
        if os.path.isdir(os.path.join(path, d)):
            path_d = path + d + '/'
            files = os.listdir(path_d)
            docs = ''
            for file in files:
                if os.path.isfile(os.path.join(path_d, file)):
                    doc = ''
                    f = open(os.path.join(path_d, file),'r')
                    for line in f:
                        if len(line)>0:
                            line = re.sub('[^a-zA-Z]', ' ', line)
                            words = []
                            for word in line.split():
                                if len(word)>1:
                                    word = word.strip().lower()
                                    words.append(word)
                            line = ' '.join(words)
                            doc = doc + " " + line
                row = [d, doc.strip()]
                docs = docs + " " + doc
                data.append(row)
            row_ = [d, docs.strip()]
            data_.append(row_)
    df_ = pd.DataFrame(data_, columns=column_names)
    df = pd.DataFrame(data, columns=column_names)
    return df_, df
                            
path_train = 'C50/C50train/'
path_test = 'C50/C50test/'   
df_train_, df_train = readfile(path_train)
df_test_, df_test = readfile(path_test)
         

In [2]:
df_train_.shape, df_train.shape, df_test_.shape, df_test.shape

((50, 2), (2500, 2), (50, 2), (2500, 2))

In [3]:
len(df_train_.iloc[0, 1]), len(df_train.iloc[0, 1]), len(df_test_.iloc[0, 1]), len(df_test.iloc[0,1])

(138599, 1912, 149858, 3593)

In [4]:
df_train_[:3]

,label,text
0,AaronPressman,the internet may be overflowing with new techn...
1,AlanCrosby,it may be only the second qualifier on the lon...
2,AlexanderSmith,financial services and property group liberty ...


In [5]:
df_train[:3]

,label,text
0,AaronPressman,the internet may be overflowing with new techn...
1,AaronPressman,the postal service announced wednesday plan to...
2,AaronPressman,elementary school students with access to the ...


In [6]:
df_test_[:3]

,label,text
0,AaronPressman,senators on tuesday sharply criticized new sec...
1,AlanCrosby,markets across eastern europe remained mired i...
2,AlexanderSmith,britain commercial property market may be impr...


In [7]:
df_test[:3]

,label,text
0,AaronPressman,senators on tuesday sharply criticized new sec...
1,AaronPressman,two members of congress criticised the federal...
2,AaronPressman,commuters stuck in traffic on the leesburg pik...


In [8]:
labels = list(df_train_['label'])
print(labels)

['AaronPressman', 'AlanCrosby', 'AlexanderSmith', 'BenjaminKangLim', 'BernardHickey', 'BradDorfman', 'DarrenSchuettler', 'DavidLawder', 'EdnaFernandes', 'EricAuchard', 'FumikoFujisaki', 'GrahamEarnshaw', 'HeatherScoffield', 'JaneMacartney', 'JanLopatka', 'JimGilchrist', 'JoeOrtiz', 'JohnMastrini', 'JonathanBirt', 'JoWinterbottom', 'KarlPenhaul', 'KeithWeir', 'KevinDrawbaugh', 'KevinMorrison', 'KirstinRidley', 'KouroshKarimkhany', 'LydiaZajc', "LynneO'Donnell", 'LynnleyBrowning', 'MarcelMichelson', 'MarkBendeich', 'MartinWolk', 'MatthewBunce', 'MichaelConnor', 'MureDickie', 'NickLouth', 'PatriciaCommins', 'PeterHumphrey', 'PierreTran', 'RobinSidel', 'RogerFillion', 'SamuelPerry', 'SarahDavison', 'ScottHillis', 'SimonCowell', 'TanEeLyn', 'TheresePoletti', 'TimFarrand', 'ToddNissen', 'WilliamKazer']


In [9]:
X_train = df_train['text']
y_train = df_train['label']
X_test = df_test['text']
y_test = df_test['label']


In [10]:
# way 1, simple SVM

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2), min_df=13)
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)


svm = LinearSVC(max_iter=10000)
prob = svm.fit(X_train_vec, y_train)
y_pred_svm = svm.predict(X_test_vec)


print("Accuracy:", accuracy_score(y_test, y_pred_svm))

Accuracy: 0.7672


In [12]:
# Accuracy: 0.7672

In [13]:
print(pd.DataFrame(confusion_matrix(y_test, y_pred_svm)).loc[10:25, 10:25])

    10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25
10  50   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11   0  47   0   0   0   0   0   0   0   0   0   0   0   0   0   0
12   0   0  23   1   2   0   0   0   0   0   4   1   0   0   0   0
13   0   0   0  27   0   0   0   0  17   0   0   0   0   0   0   0
14   0   2   0   0  19   0   0   0   0   0   0   0   0   0   0   0
15   0   0   0   0   0  50   0   0   0   0   0   0   0   0   0   0
16   0   0   0   0   0   0  42   0   0   3   0   1   0   0   1   0
17   0   0   0   0   0   0   0  35   0   0   0   0   0   0   0   0
18   0   0   0  12   0   0   0   0  35   0   0   0   0   0   0   0
19   0   0   0   0   0   0   1   0   0  44   0   0   0   0   0   0
20   0   0   0   0   0   0   0   0   0   0  50   0   0   0   0   0
21   0   0   0   0   0   0   0   0   0   0   0  48   0   0   0   0
22   0   0   0   0   0   0   0   0   0   0   0   0  35   0   0   0
23   0   0   0   0   0   0   0   0   0   0   0   0   0  47   0

In [14]:
print("normalized_mutual_info_score between predict and test:", normalized_mutual_info_score(y_test, y_pred_svm))


normalized_mutual_info_score between predict and test: 0.8103110313377363
